In [1]:
# install additional dependencies 
!pip install spacy_langdetect
!pip install contractions
!pip install pycountry
!pip install textblob
!pip install openpyxl

!pip install keras_preprocessing
!pip install keras
!pip install tensorflow
!pip install sklearn

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os
import re

from bs4 import BeautifulSoup
import pandas_profiling as pp

#Importing libraries for text pre-processing
import spacy
from spacy_langdetect import LanguageDetector
import nltk
nltk.download('stopwords')
from nltk.tokenize.toktok import ToktokTokenizer
#from nltk.tokenize import word_tokenize
from contractions import contractions_dict
import unicodedata
from typing import Dict, List

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split 


from keras.layers import Dense, Embedding, LSTM, GRU, Dropout, Bidirectional, TimeDistributed
from keras.layers import Activation, Concatenate, SpatialDropout1D, Input, Lambda, Flatten
from keras.callbacks import EarlyStopping 

from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Embedding, Flatten, Reshape
from keras.layers import Concatenate, concatenate

from keras.callbacks import ModelCheckpoint
import keras



[nltk_data] Downloading package stopwords to /Users/vinay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


In [2]:
env = "local" # colab
project_path_local = "../data/"
project_path_colab = "/content/drive/My Drive/AIML 2019 GreatLearning/Capstone Project NLP/POC"
project_path = project_path_local if env == "local" else project_path_colab
print(f"project_path: {project_path}")

project_path: ../data/


In [3]:
def combine_target_variables(row, count=3, original_target_col="Assignment group", count_col="tmp_target_count"):
        if row[count_col] <= count:
            return "OTHER"
        else:
            return row[original_target_col]

In [4]:
df_en_orig = pd.read_excel(f"{project_path}/Input Data Synthetic CleanedV3.xlsx") 

In [5]:
df_en = df_en_orig[df_en_orig['lang_textblob']=='en']
df_en.reset_index(inplace=True)
df_en["tmp_target_count"] = df_en.groupby(["Assignment group"])["Description"].transform("count") 
for index, row in df_en.iterrows():
        df_en.loc[index, "target1"] = combine_target_variables(row)
df_en.shape

(5291, 14)

In [6]:
### Paramters
df_en['description_cleaned_wc'] = df_en['description_cleaned'].apply(lambda x: len(str(x).split(" ")))
df_en['short_description_cleaned_wc'] = df_en['short_description_cleaned'].apply(lambda x: len(str(x).split(" ")))
MAX_WORDS = int(np.percentile(df_en['description_cleaned_wc'], 95))  ## based on 95 percentile
SHORT_DESC_MAX_WORDS = int(np.percentile(df_en['short_description_cleaned_wc'], 95))  ## based on 95 percentile
VALIDATION_SPLIT = 0.2 
print(f"MAX_WORDS: {MAX_WORDS}")
print(f"SHORT_DESC_MAX_WORDS: {SHORT_DESC_MAX_WORDS}")


MAX_WORDS: 85
SHORT_DESC_MAX_WORDS: 11


In [7]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(df_en.description_cleaned)
tokenizer.fit_on_texts(df_en.short_description_cleaned.apply(lambda x: f"{x}"))

word_counts = tokenizer.word_counts
word_docs = tokenizer.word_docs
word_index = tokenizer.word_index
index_word = tokenizer.index_word
document_count = tokenizer.document_count

In [8]:
no_of_descriptions = df_en.description_cleaned.size
data_shape = (no_of_descriptions, MAX_WORDS)
data = np.zeros(data_shape, dtype=np.int64)
print("data.shape", data.shape)


for description_i, description in enumerate(df_en.description_cleaned.to_list()):
    # print(f"{description_i+1} of {no_of_descriptions}")
    for word_i, word in enumerate(text_to_word_sequence(description)):
        encoded_word = word_index[word]
        if word_i >= MAX_WORDS:
            break
        elif word_i < MAX_WORDS:
            # attempt to update data only if 
            # sentence_i < MAX_SENTS and word_i < MAX_SENT_LENGTH
            data[description_i][word_i] = encoded_word

print("data[:2,:]", data[:2,:])


data.shape (5291, 85)
data[:2,:] [[ 406    6  119  142   90   73   52    6   73   18    5  189    6   33
    52  295  163   53   33    9  116    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [  66  296  104  296    1  297   31  544 3476  189  261    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]]


In [9]:
no_of_short_descriptions = df_en.short_description_cleaned.size
short_data_shape = (no_of_short_descriptions, SHORT_DESC_MAX_WORDS)
short_data = np.zeros(short_data_shape, dtype=np.int64)
print("short_data.shape", short_data.shape)


for description_i, description in enumerate(df_en.short_description_cleaned.apply(lambda x: f"{x}").to_list()):
    # print(f"{description_i+1} of {no_of_short_descriptions}")
    for word_i, word in enumerate(text_to_word_sequence(description)):
        encoded_word = word_index[word]
        if word_i >= SHORT_DESC_MAX_WORDS:
            break
        elif word_i < SHORT_DESC_MAX_WORDS:
            # attempt to update short_data only if 
            # sentence_i < MAX_SENTS and word_i < MAX_SENT_LENGTH
            short_data[description_i][word_i] = encoded_word

print("short_data[:2,:]", short_data[:2,:])
print(index_word[33], index_word[9])
print(index_word[31])

short_data.shape (5291, 11)
short_data[:2,:] [[33  9  0  0  0  0  0  0  0  0  0]
 [31  0  0  0  0  0  0  0  0  0  0]]
login issue
outlook


In [10]:
# getting unique labels in given data
labels = pd.get_dummies(df_en['target1']).values 

TARGET_LEN = len(df_en.target1.unique())
print(f"TARGET_LEN: {TARGET_LEN}")
print(df_en.target1.unique())

TARGET_LEN: 55
['GRP_0' 'GRP_1' 'GRP_3' 'GRP_4' 'GRP_5' 'GRP_8' 'GRP_6' 'GRP_10' 'GRP_9'
 'GRP_11' 'GRP_14' 'GRP_15' 'GRP_17' 'GRP_18' 'GRP_2' 'GRP_19' 'GRP_20'
 'GRP_21' 'GRP_25' 'GRP_13' 'GRP_16' 'GRP_26' 'GRP_27' 'GRP_28' 'GRP_29'
 'GRP_30' 'GRP_31' 'GRP_22' 'GRP_24' 'GRP_7' 'GRP_12' 'GRP_34' 'OTHER'
 'GRP_36' 'GRP_37' 'GRP_33' 'GRP_39' 'GRP_40' 'GRP_41' 'GRP_43' 'GRP_44'
 'GRP_45' 'GRP_46' 'GRP_47' 'GRP_23' 'GRP_50' 'GRP_42' 'GRP_51' 'GRP_52'
 'GRP_53' 'GRP_55' 'GRP_59' 'GRP_60' 'GRP_62' 'GRP_65']


In [11]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = VALIDATION_SPLIT, random_state=9) 
x_train_short, x_test_short, y_train_short, y_test_short = train_test_split(short_data, labels, test_size = VALIDATION_SPLIT, random_state=9) 
print("x_train.shape: ", x_train.shape, "  y_train.shape:", y_train.shape)
print("x_test.shape: ", x_test.shape, "  y_test.shape: ", y_test.shape) 

print("x_train_short.shape: ", x_train_short.shape, "  y_train_short.shape:", y_train_short.shape)
print("x_test_short.shape: ", x_test_short.shape, "  y_test_short.shape: ", y_test_short.shape) 

x_train.shape:  (4232, 85)   y_train.shape: (4232, 55)
x_test.shape:  (1059, 85)   y_test.shape:  (1059, 55)
x_train_short.shape:  (4232, 11)   y_train_short.shape: (4232, 55)
x_test_short.shape:  (1059, 11)   y_test_short.shape:  (1059, 55)


In [12]:
### trying with regularizing embedding

LSTM_DIM = MAX_WORDS # 128
OUTPUT_LEN = 150 # embedding_dimentations
vocab_size = len(tokenizer.word_index.keys()) + 1

description_encoder_inputs = Input(shape=(MAX_WORDS,))

# x1 = Reshape((400,))(description_encoder_inputs)
x2 = Embedding(
        output_dim=OUTPUT_LEN, 
        input_dim=vocab_size, 
        input_length=MAX_WORDS, 
        embeddings_regularizer=keras.regularizers.l2(.001))(description_encoder_inputs)

description_encoder = Bidirectional(LSTM(OUTPUT_LEN, dropout=0.25, recurrent_dropout=0.25)) # return_state=True
description_encoder_outputs = description_encoder(x2) # description_state_h, description_state_c


short_description_encoder_inputs = Input(shape=(SHORT_DESC_MAX_WORDS,))

# sx1 = Reshape((400,))(short_description_encoder_inputs)
sx2 = Embedding(
        output_dim=OUTPUT_LEN, 
        input_dim=vocab_size, 
        input_length=SHORT_DESC_MAX_WORDS, 
        embeddings_regularizer=keras.regularizers.l2(.001))(short_description_encoder_inputs)

short_description_encoder = Bidirectional(LSTM(OUTPUT_LEN, dropout=0.25, recurrent_dropout=0.25)) # return_state=True
short_description_encoder_outputs = short_description_encoder(sx2) # description_state_h, description_state_c

combined_context_vector = concatenate([description_encoder_outputs, short_description_encoder_outputs])

d1 = Dense(units = 256, activation='relu')(combined_context_vector) # combined_context_vector
d2 = Dropout(0.2)(d1)
predictions = Dense(TARGET_LEN, activation='softmax')(d2) 

bidirectional_lstm_model = Model(inputs=[description_encoder_inputs, short_description_encoder_inputs], outputs=predictions)
bidirectional_lstm_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


print(bidirectional_lstm_model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 85)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 11)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 85, 150)      1440150     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 11, 150)      1440150     input_2[0][0]                    
____________________________________________________________________________________________

In [14]:
%%time
# batch_size = 32
# model.fit(x_train_re, y_train, epochs=5, batch_size=batch_size, verbose=2, validation_split=VALIDATION_SPLIT)

batch_size = 64
epochs = 10
# filepath = "saved-model-{epoch:02d}-{val_acc:.2f}.hdf5"
# filepath = "saved-model-{epoch:02d}-{val_accuracy:.2f}.hdf5"
filepath = "saved_models/bidirectional-lstm-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=False, mode='auto')

print('Training...')

bidirectional_lstm_model.fit([x_train, x_train_short], y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=VALIDATION_SPLIT,
          callbacks=[checkpoint]) 

Training...
Train on 3385 samples, validate on 847 samples
Epoch 1/10
3385/3385 [==============================] - 60s 18ms/step - loss: 1.8652 - accuracy: 0.5666 - val_loss: 1.9937 - val_accuracy: 0.5443

Epoch 00001: saving model to saved_models/bidirectional-lstm-01-0.54.hdf5
Epoch 2/10
3385/3385 [==============================] - 57s 17ms/step - loss: 1.6841 - accuracy: 0.6035 - val_loss: 1.9880 - val_accuracy: 0.5632

Epoch 00002: saving model to saved_models/bidirectional-lstm-02-0.56.hdf5
Epoch 3/10
3385/3385 [==============================] - 52s 15ms/step - loss: 1.5380 - accuracy: 0.6281 - val_loss: 1.9920 - val_accuracy: 0.5596

Epoch 00003: saving model to saved_models/bidirectional-lstm-03-0.56.hdf5
Epoch 4/10
3385/3385 [==============================] - 56s 17ms/step - loss: 1.4550 - accuracy: 0.6496 - val_loss: 2.0379 - val_accuracy: 0.5714

Epoch 00004: saving model to saved_models/bidirectional-lstm-04-0.57.hdf5
Epoch 5/10
3385/3385 [==============================] - 5

In [15]:
%%time
# evaluating model on validation data set
 
loss, acc = bidirectional_lstm_model.evaluate([x_test, x_test_short], y_test, batch_size=batch_size)
print('Test loss:', loss)
print('Test accuracy:', acc)

1059/1059 [==============================] - 4s 4ms/step
Test loss: 2.415554674421424
Test accuracy: 0.585457980632782
CPU times: user 9.15 s, sys: 1.12 s, total: 10.3 s
Wall time: 4.43 s


In [16]:
y_pred = bidirectional_lstm_model.predict([x_test, x_test_short], verbose=1)
y_pred_index = np.argmax(y_pred, axis=1)
y_pred_class = (y_pred == y_pred.max(axis=1, keepdims=True)).astype(int)
print(f"y_pred_class: {y_pred_class[1]}")



1059/1059 [==============================] - 6s 5ms/step
y_pred_class: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [17]:
from sklearn.metrics import classification_report
print("y_pred_class: ", y_pred_class[:1])
print("y_test: ", y_test[:1])
print(classification_report(y_test, y_pred_class, target_names=df_en.target1.unique()))

y_pred_class:  [[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
y_test:  [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
              precision    recall  f1-score   support

       GRP_0       0.75      0.87      0.80       511
       GRP_1       0.00      0.00      0.00         6
       GRP_3       0.06      0.10      0.07        10
       GRP_4       0.00      0.00      0.00         5
       GRP_5       0.57      0.41      0.48        29
       GRP_8       0.55      0.43      0.48        28
       GRP_6       0.38      0.47      0.42        17
      GRP_10       0.00      0.00      0.00         5
       GRP_9       0.40      0.21      0.28        19
      GRP_11       0.00      0.00      0.00         4
      GRP_14       0.22      0.14      0.17        14
      GRP_15       0.18      0.17      0.17        36
      GRP_17       0.44      0.38      0.